In [14]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
dataset_dir = '/content/drive/MyDrive/Projeto Bimestral/DataSet/Treino'

fruits_classes = ['Acai', 'Acerola', 'Ackee', 'Apple', 'Araza', 'Avocado', 'Bael', 'Banana', 'BayBerry', 'BlueBerry']

target_size = (224, 224)
images = []
labels = []

for fruit_class in fruits_classes:
    class_dir = os.path.join(dataset_dir, fruit_class)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, target_size)
        image = image.astype('float32') / 255.0
        images.append(image)
        labels.append(fruits_classes.index(fruit_class))

In [3]:
images = np.array(images)
labels = np.array(labels)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [4]:
num_classes = len(fruits_classes)
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

In [6]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=32, epochs=2, validation_data=(test_images, test_labels))

Epoch 1/2
71/71 [==============================] - 849s 12s/step - loss: 2.2568 - accuracy: 0.2848 - val_loss: 1.6013 - val_accuracy: 0.4190
Epoch 2/2
71/71 [==============================] - 843s 12s/step - loss: 1.3888 - accuracy: 0.5062 - val_loss: 1.4464 - val_accuracy: 0.4560


In [7]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


18/18 [==============================] - 54s 3s/step - loss: 1.4464 - accuracy: 0.4560
Test Loss: 1.4463659524917603
Test Accuracy: 0.4559859037399292


In [16]:
model.save('vgg_model.h5')

In [19]:
def preprocess_single_image(image, target_size):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (target_size, target_size))
    image = image.astype("float32") / 255.0
    return image

# Caminho da nova imagem
imagem_nova = '/content/drive/MyDrive/Projeto Bimestral/DataSet/imagem_nova/BlueBerry_320.jpg'

# Carregar a nova imagem
nova_imagem = cv2.imread(imagem_nova)

# Dimensão desejada para a imagem
target_size = 224

# Pré-processar a nova imagem
nova_imagem = preprocess_single_image(nova_imagem, target_size)

# Expandir as dimensões da imagem para corresponder às dimensões esperadas pelo modelo
nova_imagem = np.expand_dims(nova_imagem, axis=0)

# Carregar o modelo salvo
loaded_model = tf.keras.models.load_model('vgg_model.h5')

# Fazer a previsão com o modelo carregado
predictions = loaded_model.predict(nova_imagem)
predicted_class_index = np.argmax(predictions)
predicted_class = fruits_classes[predicted_class_index]

print("Classe prevista:", predicted_class)

1/1 [==============================] - 0s 453ms/step
Classe prevista: BlueBerry
